<a href="https://colab.research.google.com/github/eba57/hatespeechdetection/blob/main/amharicRANDOMFOREST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.utils import np_utils
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn import tree
from tensorflow.keras.preprocessing import sequence
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import files
data_to_load = files.upload()
import io
dataset = pd.read_csv(io.BytesIO(data_to_load['amharic.csv']))

Saving amharic.csv to amharic.csv


In [3]:
tweet=dataset.tweet
labels=dataset.labels

In [4]:
dataset['labels']=[re.sub(r"\n", "", str(label)) for label in labels]

In [5]:
# drop function which is used in removing or deleting rows or columns from the CSV files
dataset.drop('Unnamed: 2', inplace=True, axis=1)

# display
print("\nCSV Data after deleting the column 'Unnamed: 2':\n")
print(dataset)


CSV Data after deleting the column 'Unnamed: 2':

      labels                                              tweet
0      Free   አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1       Free   እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2      Free   የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3       Free  ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4       Hate  ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...
...      ...                                                ...
29995   Hate         በአሉ የሁሉም ኢትዮጵያዊ ስላልሆነ በኦሮምኛው ቢለፋደድ ምን አገባን
29996   Free             ተባረክ አብቹ ፈር ቀዳጅ ስለሆንህ መጋረጃው መቀደድ ስለጀመረ
29997   Free   እስከ አሁን አንተ ብቻ ነው በ   መፅሀፍ ያልቻልከው  አንተም ታሪክ እ...
29998   Hate   ህገወጥት ጠቅላይ ሚንስትር ፅቤት የተፈቀደ ሆኖ ህዝብን እንዴት ህግ አክ...
29999   Hate  ደነዙ ጠቅላይ ሚንስትር ፅቤት ህገመንግስት ሳይሻሻል በህግ የተወሰነዉን የ...

[30000 rows x 2 columns]


In [6]:
words = dataset['tweet']
finalWords = []
for word in words:
    
    
    #remove any non-alphanumeric character
    word = re.sub(r'\W', ' ', str(word))
    
    #remove any digit
    word = re.sub("(\\d)+"," ",word)
    
    #remove punctuation 
    word = re.sub(",#'!:;"," ",word)
    
    #remove any latin characters 
    word = re.sub(r'[a-zA-Z]+', '', word)

    word = re.sub('\s+', ' ', word)  # remove extra whitespace
    finalWords.append(word)

In [7]:
tokenizedList = []
for lineWord in finalWords:
    separatedWords = lineWord.split(' ')
    word = [word for word in separatedWords if   word.strip() != '']
    tokenizedList.append(word)

In [8]:
# list amharic stop words
stopwords=['ነው', 'እና', 'እኔ', 'የእኔ','እኔ','ራሴ','እኛ','የእኛ','እኛ','ራሳችን','አንቺ' ,'የእርስዎ' ,'እራስዎ','እራሳችሁን','የእሱ',
           'እሱ','እ','ራሱ','እሷ' 'ራሷን','እሱ','በራሱ','እነሱ','የእነሱ','እራሳቸው','ምንድን','የት','ማን','ይሄ','ያንን','እነዚህ','እነዚያ'
           ,'ነው','እነዚህ','ናቸው','ነበር','ነበሩ','መሆን','ሆኗል','መሆን','አለ','ነበሩ','አለው','መስራት','ያደርገዋል','አደረጉት',
           'ማድረግ','እና','ግን','ከሆነ','ወይም','ምክንያቱም','እንደ','እስከ','ገና','የ','ለ','ከ','ጋር','ስለ','መካከል','ውስጥ',
           'ከዚህ','በፊት','በኋላ','ከላይ','ከታች','ወደ','ከ','ወደ','ላይ','ታች','ውስጥ','ውጭ','በላይ','በታች','በድጋሚ','እናስ','እናም']

           
cleanedWordsList= []
for lineWords in tokenizedList:
    tempList = []
    for word in lineWords:
        if word.strip() not in stopwords:
            tempList.append(word)
    cleanedWordsList.append(tempList)

In [9]:
# for the time being this is fine but convert it to something good but not this at all

normalizedSenteces = []
for linesWords in cleanedWordsList:
    tempList = []
    for word in linesWords:
        
        word = re.sub("ሐ","ሀ",word)
        word = re.sub("ሑ","ሁ",word)
        word = re.sub("ሒ","ሂ",word)
        word = re.sub("ሓ","ሃ",word)
        word = re.sub("ሔ","ሄ",word)
        word = re.sub("ሕ","ህ",word)
        word = re.sub("ሖ","ሆ",word)
        
        word = re.sub("ኀ","ሀ",word)
        word = re.sub("ኁ","ሁ",word)
        word = re.sub("ኂ","ሂ",word)
        word = re.sub("ኃ","ሃ",word)
        word = re.sub("ኄ","ሄ",word)
        word = re.sub("ኅ","ህ",word)
        word = re.sub("ኆ","ሆ",word)
        
        word = re.sub("ሠ","ሰ",word)
        word = re.sub("ሡ","ሱ",word)
        word = re.sub("ሢ","ሲ",word)
        word = re.sub("ሣ","ሳ",word)
        word = re.sub("ሤ","ሴ",word)
        word = re.sub("ሥ","ስ",word)
        word = re.sub("ሦ","ሶ",word)
        
        word = re.sub("ዐ","አ",word)
        word = re.sub("ዑ","ኡ",word)
        word = re.sub("ዒ","ኢ",word)
        word = re.sub("ዓ","ኣ",word)
        word = re.sub("ዔ","ኤ",word)
        word = re.sub("ዕ","እ",word)
        word = re.sub("ዖ","ኦ",word)
        
        word = re.sub("ዐ","አ",word)
        word = re.sub("ዑ","ኡ",word)
        word = re.sub("ዒ","ኢ",word)
        word = re.sub("ዓ","ኣ",word)
        word = re.sub("ዔ","ኤ",word)
        word = re.sub("ዕ","እ",word)
        word = re.sub("ዖ","ኦ",word)
        
        word = re.sub("ጸ","ፀ",word)
        word = re.sub("ጹ","ፁ",word)
        word = re.sub("ጺ","ፂ",word)
        word = re.sub("ጻ","ፃ",word)
        word = re.sub("ጼ","ፄ",word)
        word = re.sub("ጽ","ፅ",word)
        word = re.sub("ጾ","ፆ",word)
        
        tempList.append(word)
    normalizedSenteces.append(tempList)

In [10]:
# # Split into training and testing data
x = dataset['tweet']
y = dataset['labels']

In [11]:
normalizedSenteces  = pd.Series(normalizedSenteces)

finalSentences = []
for lists in normalizedSenteces:
    temp = ""
    for word in lists:
        temp =  temp +" "+word
    finalSentences.append(temp)

    
x = pd.Series(finalSentences)

In [13]:


# initial TTS
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=15)



In [15]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [17]:
from nltk.corpus import stopwords
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [19]:
# instantiating tf_idf vectorizer
tfidf = TfidfVectorizer( ngram_range=(1,2))

In [20]:
# transforming tokenized data into sparse matrix format with 20K stored elements
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [21]:
# taking a quick look at the non zero elements
non_zero_cols = X_train_tfidf.nnz / float(X_train_tfidf.shape[0])
print("Average Number of Non-Zero Elements in Vectorized Articles: {}".format(non_zero_cols))
percent_sparse = 1 - (non_zero_cols / float(X_train_tfidf.shape[1]))
print('Percentage of columns containing ZERO: {}'.format(percent_sparse))

Average Number of Non-Zero Elements in Vectorized Articles: 31.093958333333333
Percentage of columns containing ZERO: 0.9999196708750773


In [22]:


rf_baseline = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=0)



In [23]:


%%time
rf_baseline.fit(X_train_tfidf, y_train)
rf_test_preds = rf_baseline.predict(X_test_tfidf)



CPU times: user 1h 26min 7s, sys: 5.64 s, total: 1h 26min 13s
Wall time: 1h 26min 29s


In [56]:

 #rf_baseline = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,criterion='gini' ,random_state=42)
            # Train the model
            
           #train and test
#from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# If you're working in Jupyter Notebook, include the following so that plots will display:
#%matplotlib inline
#accuracy_score(y_test, rf_test_preds)

#score = (rf_baseline.score(X_test, y_test)*100)
           
            # Print the results
#print(score)


0.6741666666666667

In [57]:
#precision_score(y_test, rf_test_preds)


TypeError: ignored

In [72]:
#from sklearn.metrics import f1_score
#print("Precision Score : ",precision_score(y_test, rf_test_preds, pos_label='positive' average='micro'))
#f1 = f1_score(Y_test, Y_pred)
#print("Recall Score : ",recall_score(y_test, rf_test_preds,pos_label='positive'average='micro'))

In [71]:
# View the classification report for test data and predictions
#print(classification_report(y_test, rf_test_preds))
#print(confusion_matrix(y_test, rf_test_preds))